In [1]:
# 또는 상위 폴더에 db가 있다면
import sys
sys.path.append('..')


In [2]:
import mysql.connector
from mysql.connector import Error

def test_database_connection(db_config):
    """데이터베이스 연결 테스트"""
    try:
        print("데이터베이스 연결 테스트 중...")
        conn = mysql.connector.connect(**db_config)
        
        if conn.is_connected():
            cursor = conn.cursor()
            
            # trading_signals 테이블 확인
            cursor.execute("SHOW TABLES LIKE 'trading_signals'")
            if not cursor.fetchone():
                print("❌ 'trading_signals' 테이블이 존재하지 않습니다.")
                print("   trading_signals.sql을 먼저 실행하세요.")
                cursor.close()
                conn.close()
                return False
            
            print("✅ 데이터베이스 연결 성공")
            print("✅ 'trading_signals' 테이블 확인 완료")
            
            cursor.close()
            conn.close()
            return True
            
    except Error as e:
        print(f"❌ 데이터베이스 연결 실패: {e}")
        return False


def update_supported_symbols(db_config, symbols):
    """지원 종목을 DB에 업데이트"""
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        
        # supported_symbols 테이블 존재 확인
        cursor.execute("SHOW TABLES LIKE 'supported_symbols'")
        if not cursor.fetchone():
            print("⚠️  'supported_symbols' 테이블이 없습니다. (선택사항, 스킵)")
            cursor.close()
            conn.close()
            return
        
        for symbol in symbols:
            try:
                # 바이낸스 심볼로 변환
                binance_mapping = {
                    'BTC': 'BTCUSDT', 'ETH': 'ETHUSDT', 'SHIB': '1000SHIBUSDT',
                    'DOT': 'DOTUSDT', 'BCH': 'BCHUSDT', 'AVAX': 'AVAXUSDT',
                    'LINK': 'LINKUSDT', 'ADA': 'ADAUSDT', 'AAVE': 'AAVEUSDT',
                    'TAO': 'TAOUSDT', 'SEI': 'SEIUSDT'
                }
                
                binance_symbol = binance_mapping.get(symbol, f"{symbol}USDT")
                
                # INSERT OR UPDATE
                query = """
                INSERT INTO supported_symbols (symbol, base_asset, quote_asset, is_active)
                VALUES (%s, %s, 'USDT', TRUE)
                ON DUPLICATE KEY UPDATE is_active = TRUE
                """
                
                cursor.execute(query, (binance_symbol, symbol))
                
            except Error as e:
                print(f"⚠️  {symbol} 업데이트 실패: {e}")
        
        conn.commit()
        print(f"✅ {len(symbols)}개 종목 DB 업데이트 완료")
        
        cursor.close()
        conn.close()
        
    except Error as e:
        print(f"❌ 종목 업데이트 실패: {e}")

In [ ]:
import schedule
import time
from ai_trading_signal import *
if __name__ == "__main__":
    
    # ===== 설정 =====
    MODEL_PATH = r'C:\Users\soekjun\Desktop\AI모델\projects\coin\models\v01\crypto_xgboost\crypto_xgboost_final_model.pkl'
    
    DB_CONFIG = {
        'host': 'localhost',
        'port': 3306,
        'user': 'root',
        'password': 'ghkd2759A!',
        'database': 'trading_signals'
    }
    
    SYMBOLS = [
        'BTC', 'ETH', 'SHIB', 'DOT', 'BCH', 'AVAX',
        'LINK', 'ADA', 'AAVE', 'TAO', 'SEI'
    ]
    
    # ===== 초기화 =====
    print("\n" + "="*70)
    print("시스템 초기화")
    print(f"감시 종목: {len(SYMBOLS)}개")
    print("="*70)
    
    if not test_database_connection(DB_CONFIG):
        print("데이터베이스 연결 실패. 프로그램을 종료합니다.")
        exit(1)
    
    print("\n지원 종목 업데이트 중...")
    update_supported_symbols(DB_CONFIG, SYMBOLS)
    
    # ===== 시스템 시작 =====
    print("\n" + "="*70)
    print("자동 트레이딩 시그널 시스템 시작")
    print(f"감시 종목: {', '.join(SYMBOLS)}")
    print("="*70)
    
    try:
        system = AutoTradingSignal(MODEL_PATH, DB_CONFIG)
        
        # 1. 스크립스 실행시 즉시 1회 실행
        print("\n🚀 초기 예측 실행...\n")
        system.run_once()
        
        # 2. 스케줄 등록 (09:00, 17:00, 01:00)
        print("\n📅 스케줄러 시작: 09:00, 17:00, 01:00에 자동 실행")
        print("📌 Ctrl+C를 눌러 중단할 수 있습니다.\n")
        
        schedule.every().day.at("09:00").do(system.run_prediction_all)
        schedule.every().day.at("17:00").do(system.run_prediction_all)
        schedule.every().day.at("01:00").do(system.run_prediction_all)
        
        # 3. 매일 무한 루프
        while True:
            schedule.run_pending()
            time.sleep(60)  # 1분마다 체크
            
    except KeyboardInterrupt:
        print("\n\n사용자에 의해 중단되었습니다.")
        print("프로그램을 종료합니다.")

    except Exception as e:
        print(f"\n시스템 오류: {e}")
        import traceback
        traceback.print_exc()
        print("프로그램을 종료합니다.")


시스템 초기화
감시 종목: 11개
데이터베이스 연결 테스트 중...
✅ 데이터베이스 연결 성공
✅ 'trading_signals' 테이블 확인 완료

지원 종목 업데이트 중...
✅ 11개 종목 DB 업데이트 완료

자동 트레이딩 시그널 시스템 시작
감시 종목: BTC, ETH, SHIB, DOT, BCH, AVAX, LINK, ADA, AAVE, TAO, SEI
모델 로드 완료: C:\Users\soekjun\Desktop\AI모델\projects\coin\models\v01\crypto_xgboost\crypto_xgboost_final_model.pkl
자동 트레이딩 시그널 시스템 초기화 완료
감시 종목 (11개): BTC, ETH, SHIB, DOT, BCH, AVAX, LINK, ADA, AAVE, TAO, SEI

🚀 초기 예측 실행...


🧪 단일 실행 모드


🤖 AI 예측 사이클 시작: 2025-10-02 12:04:05

📊 BTC 분석 중...

🔄 BTC 전처리 시작
✅ BTC: 120일 데이터 수집 완료
🔧 기술 지표 계산 중...
🎯 주요 피처 선택 중...
📊 정규화 적용 중...
✂️  최종 피처 필터링 중...
✅ 전처리 완료: 28개 피처

✅ BTC: 1일 데이터 수집 완료
BTC -> BTCUSDT: SHORT (신뢰도: 73.58%)
   ✅ DB 업데이트: BTCUSDT

📊 ETH 분석 중...

🔄 ETH 전처리 시작
✅ ETH: 120일 데이터 수집 완료
🔧 기술 지표 계산 중...
🎯 주요 피처 선택 중...
📊 정규화 적용 중...
✂️  최종 피처 필터링 중...
✅ 전처리 완료: 28개 피처

✅ ETH: 1일 데이터 수집 완료
ETH -> ETHUSDT: HOLD (신뢰도: 53.14%)
   ✅ DB 업데이트: ETHUSDT

📊 SHIB 분석 중...

🔄 SHIB 전처리 시작
✅ SHIB: 120일 데이터 수집 완료
🔧 기술 지표 계산 중...
🎯 주요 피처 선택 중...
📊 정규화 적용 중...


# 서버 시간대 확인

In [1]:
"""
서버 시간대 확인 노트북
"""
from datetime import datetime
import pytz
import subprocess

print("="*60)
print("🕐 서버 시간대 확인")
print("="*60)

# 1. 현재 서버 시간
print(f"\n📅 현재 서버 시간: {datetime.now()}")
print(f"📅 현재 UTC 시간: {datetime.utcnow()}")

# 2. 시간 차이 계산
time_diff = (datetime.now() - datetime.utcnow()).total_seconds() / 3600
print(f"\n⏰ UTC와의 시간 차이: {time_diff:+.1f}시간")

if abs(time_diff) < 0.5:
    print("   → 서버는 UTC 시간대입니다")
    print("   → 한국시간(KST) 09:00 = UTC 00:00")
    print("   → start_hour=0 으로 설정하세요!")
elif abs(time_diff - 9) < 0.5:
    print("   → 서버는 한국 시간대(KST)입니다")
    print("   → start_hour=9 그대로 사용하세요!")
else:
    print(f"   → 서버는 UTC{time_diff:+.1f} 시간대입니다")

# 3. 한국시간 표시
kst = pytz.timezone('Asia/Seoul')
print(f"\n🇰🇷 현재 한국시간: {datetime.now(kst).strftime('%Y-%m-%d %H:%M:%S %Z')}")

# 4. 시스템 시간대 정보
try:
    result = subprocess.run(['timedatectl'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"\n🖥️  시스템 시간대 정보:\n{result.stdout}")
except:
    print("\n⚠️  timedatectl 명령어를 사용할 수 없습니다.")

print("="*60)

🕐 서버 시간대 확인

📅 현재 서버 시간: 2025-09-29 12:40:01.590646
📅 현재 UTC 시간: 2025-09-29 03:40:01.590646

⏰ UTC와의 시간 차이: +9.0시간
   → 서버는 한국 시간대(KST)입니다
   → start_hour=9 그대로 사용하세요!

🇰🇷 현재 한국시간: 2025-09-29 12:40:03 KST

⚠️  timedatectl 명령어를 사용할 수 없습니다.
